# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770395803717                   -0.53    8.0         
  2   -2.771686110386       -2.89       -1.31    1.0    163ms
  3   -2.771714307745       -4.55       -2.53    1.0    182ms
  4   -2.771714667458       -6.44       -3.45    1.0    183ms
  5   -2.771714713299       -7.34       -3.90    2.0    158ms
  6   -2.771714715180       -8.73       -5.04    1.0    158ms
  7   -2.771714715245      -10.19       -5.20    2.0    163ms
  8   -2.771714715250      -11.31       -6.31    1.0    173ms
  9   -2.771714715250      -12.84       -6.87    2.0    185ms
 10   -2.771714715250   +  -15.05       -7.26    1.0    161ms
 11   -2.771714715250      -13.94       -8.11    1.0    150ms


-0.00013457284494529376

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770298019209                   -0.53    9.0         
  2   -2.771770543953       -2.83       -1.30    1.0    158ms
  3   -2.771801439045       -4.51       -2.66    1.0    217ms
  4   -2.771802072400       -6.20       -3.97    2.0    153ms
  5   -2.771802074410       -8.70       -4.62    2.0    175ms
  6   -2.771802074473      -10.20       -5.46    1.0    180ms
  7   -2.771802074476      -11.47       -6.49    2.0    159ms
  8   -2.771802074476      -13.94       -7.38    2.0    197ms
  9   -2.771802074476   +  -14.75       -8.04    1.0    167ms


0.017612220806449357

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457284494529376
Displaced dipole:  0.017612220806449357
Polarizability :   1.774679365139465


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920630804e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551002185e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852762332585e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593782428e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787702913e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.275619145316e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.411954055755e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.411954055755e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.747992990479e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.265368769394e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.911310488895e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.409619717020e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.409619717020e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.